In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
from tickers500 import tickers500
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1
from recommendations import Recommendations
from moonShot2 import MoonShot2
import streamlit as st

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 


In [2]:
rec = Recommendations()
moonShot2 = MoonShot2(rec.buytable_df)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=institutionOwnership%2CfundOwnership%2CmajorDirectHolders%2CmajorHoldersBreakdown%2CinsiderTransactions%2CinsiderHolders%2CnetSharePurchaseActivity&corsDomain=finance.yahoo.com&symbol=MMM&formatted=false&crumb=ffP1U80oFA.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=institutionOwnership%2CfundOwnership%2CmajorDirectHolders%2CmajorHoldersBreakdown%2CinsiderTransactions%2CinsiderHolders%2CnetSharePurchaseActivity&corsDomain=finance.yahoo.com&symbol=AOS&formatted=false&crumb=ffP1U80oFA.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=institutionOwnership%2CfundOwnership%2CmajorDirectHolders%2CmajorHoldersBreakdown%2CinsiderTransactions%2CinsiderHolders%2CnetSharePurchaseActivity&corsDomain=finance.yahoo.com&symbol=ABT&formatted=false&crumb=ffP1U80oFA.
404 Client Error: Not 

KeyError: 'Signal'

In [ ]:

print(rec.signals_df)

In [ ]:
# rec.buy_recommendations_df.to_csv('CSV/buy_recommendations.csv')
rec.buy_recommendations_df


In [ ]:
# rec.sell_recommendations_df.to_csv('CSV/sell_recommendations.csv')
rec.sell_recommendations_df


In [ ]:
rec.tradingstrategy1_df.to_csv('CSV/tradingstrategy1.csv')
rec.tradingstrategy1_df


---
# Further research
---

### Below we retrieve individual stock data based on it appearing in the buy/sell list above

In [ ]:

CSX = Ticker('CSX', rec.start, rec.end)
CSX_df = CSX.df
CSX_df

CSX.profitability


In [ ]:
CSX.piotroski


In [ ]:
CSX.ratios